## ライブラリのインポート
評価に必要な主要ライブラリを読み込み

In [ ]:
import pandas as pd
import numpy as np
import joblib
import json
import re
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

## パスの設定
モデル・データ・エンコーダのパスを定義

In [ ]:
BASE_DIR = Path("/usr/src/app/backend")
data_path = BASE_DIR / "data" / "raw" / "test-data.csv"
model_path = BASE_DIR / "ml" / "models" / "ml_model.pkl"

## テストデータの読み込み

In [ ]:
df = pd.read_csv(data_path)

## 不要列の削除
分析に不要な列を削除

In [ ]:
df = df.drop(columns=["Unnamed: 0", "Name", "New_Price"], errors='ignore')

## 単位除去と数値変換
'kmpl', 'CC', 'bhp'などの単位を除去し数値型に変換

In [ ]:
columns_with_units = ['Mileage', 'Engine', 'Power']
for col in columns_with_units:
    df[col] = df[col].astype(str).str.extract(r'([\d.]+)')[0]
    df[col] = pd.to_numeric(df[col], errors='coerce')

## 無効なレコードの除外
Mileageが0のレコードを除外

In [ ]:
df = df[df['Mileage'] != 0]

## 欠損値の補完
数値列の欠損を中央値で補完

In [ ]:
for col in ['Mileage', 'Engine', 'Power', 'Seats']:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

## カテゴリ列の指定
モデル学習時に使用したカテゴリ列を指定

In [ ]:
categorical_columns = ['Fuel_Type', 'Transmission', 'Owner_Type', 'Location']

## モデルの読み込み

In [ ]:
model = joblib.load(model_path)

## カテゴリ変数のエンコーディング
保存済エンコーダ（OneHotEncoder）を使用してカテゴリ変数をエンコード

In [ ]:
encoder_path = BASE_DIR / "ml" / "models" / "ml_encoder.pkl"
ct = joblib.load(encoder_path)
X = ct.transform(df)

## 予測の実行

In [ ]:
y_pred = model.predict(X)

## 予測結果のデータフレーム作成

In [ ]:
result_df = df.copy()
result_df['Predicted_Price'] = y_pred

## 予測結果のヒストグラム表示

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(y_pred, bins=30, edgecolor='black', alpha=0.7)
plt.title("Predicted Price Distribution")
plt.xlabel("Price")
plt.ylabel("Frequency")
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 予測結果の表示

In [ ]:
pd.set_option('display.float_format', '{:,.0f}'.format)
display(result_df[['Location', 'Year', 'Fuel_Type', 'Transmission', 'Owner_Type',
                   'Engine', 'Power', 'Mileage', 'Seats', 'Predicted_Price']])